# Synteza programów z NL

## Przygotowanie tensorflow

In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Przygotowanie zbioru danych.

Klasa Dataset odpowieada za wczytanie zbiorów danych z plików oraz
przygotowanie ich do podania na wejście modelu, a także dekodowania wyjścia
do postacji wykonywalnego programu i jego uruchomiania oraz testowania.
Ostatni parametr konstruktora odpowiada za rodzaj reprezentacji programów:
- True -  bez nawiasów,
- False - z nawiasami.

In [2]:
from dataset import DataSet
dataset = DataSet(10, 80_000, 10_000, True)

Loading programs from jsonl file (filtered_data/metaset3.train.jsonl): 100%|██████████| 74138/74138 [00:04<00:00, 16786.46it/s]
Preparing training dataset: 100%|██████████| 74138/74138 [00:00<00:00, 138514.26it/s]
Loading programs from jsonl file (filtered_data/metaset3.dev.jsonl): 100%|██████████| 10226/10226 [00:00<00:00, 35204.80it/s]
Preparing validation dataset: 100%|██████████| 10226/10226 [00:00<00:00, 144850.00it/s]
Loading programs from jsonl file (filtered_data/metaset3.test.jsonl): 100%|██████████| 9405/9405 [00:00<00:00, 12156.04it/s]
Preparing test dataset: 100%|██████████| 9405/9405 [00:00<00:00, 126060.73it/s]


Programs size: (74138, 219)
Test count: 724359
Programs size: (10226, 219)
Test count: 100329
Programs size: (9405, 219)
Test count: 91758


## Wczytanie modelu

Pierwszy parametr modelu to długość embeding'u, drugi odpowiada za liczbę jednostek
w każdej warstwie modelu. Ostatnim parametrem konsturktora jest wcześniej przygotowany
zbiór danych.

In [3]:
from model import Seq2Seq
model = Seq2Seq(128, 320, dataset)

Input Vocab size: 276
Output Vocab size: 73


## Szczegóły modelu

In [4]:
model.write_summary()

Model: "Encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 151)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 151, 128)     35328       input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(10, 151, 320), (10 432000      embedding[0][0]                  
Total params: 467,328
Trainable params: 467,328
Non-trainable params: 0
____________________

## Wczytanie wag modelu

Przy pomocy metody load_last zostaną wczytane wagi z nalepszego momentu
treningu.

In [5]:
model.load_last("17-12-2020")

## Odpytanie modelu

Poniżej przedstawiony jest uproszczony przebieg odpytania modelu od momentu
wczytania danych do momentu testowania wygenerowanego programu na przypadku testowym.

Przykładowe działające numery programów: 13, 14, 17, 84, 153...,
aby przetestować inny przykład wystarczy zmienić wartość zmiennej **program_num**.

In [6]:
import os
from DataLoader import load_programs_json, tokenize_program

# Wczytanie danych z pliku
programs = load_programs_json(os.path.join("filtered_data", "metaset3.test.jsonl"))

program_num = 14
text = programs['text'][program_num]
print(f"Opis programu: {text}")
print(f"Argumenty programu: {programs['args'][program_num]}")
print(f"Typ zwracany przez program: {programs['return_type'][program_num]}")

# Odpytanie modelu
code, encoded_program = model.evaluate_sentence(text)
# Tokenizacja argumentów programu
_, program_args = tokenize_program(programs['short_tree'][program_num], programs['args'][program_num])
program, args = dataset.decode_program(tf.convert_to_tensor(encoded_program), tf.convert_to_tensor(program_args))
print(f"Wygenerowany kod: {code.replace('<end>', '')}  (przed wstawieniem nawiasów)")
print(f"Oczekiwany kod: {programs['short_tree'][program_num]}")
print(f"Wejście przypadku tesotwego: {programs['tests'][program_num][0]['input']}")
test_input = programs['tests'][program_num][0]["input"]

# "Skompilowanie" wygenerowanego programu do funkcji
func = dataset.compile_func(program, args, programs['return_type'][program_num])

# Przygotowanie wejście programu
test_args = [test_input[a] for a in test_input.keys()]
output = func(*test_args)
print(f"Zwrócone wyjście: {output}")
print(f"Oczekiwane wyjście: {programs['tests'][program_num][0]['output']}")

Loading programs from jsonl file (filtered_data/metaset3.test.jsonl): 100%|██████████| 9405/9405 [00:00<00:00, 34157.05it/s]


Opis programu: consider an array of numbers a , compute elements of a that are odd
Argumenty programu: {'a': 'int[]'}
Typ zwracany przez program: int[]
Wygenerowany kod: filter a lambda1 == % arg1 2 1   (przed wstawieniem nawiasów)
Oczekiwany kod: ['filter', 'a', ['lambda1', ['==', ['%', 'arg1', '2'], '1']]]
Wejście przypadku tesotwego: {'a': [2, 9, 9, 17, 4, 6, 30]}
Zwrócone wyjście: [9, 9, 17]
Oczekiwane wyjście: [9, 9, 17]
